# Data Generation

### Image - Fixed Foveation

In [19]:
import numpy as np
np.random.seed(19980430)
import cv2
from scipy import signal

In [20]:
inset_x = 0.5
inset_y = 0.5
inset_half = 0.125
blur = 4  # scale of blurring for periphery

In [21]:
def blur_image (im, inset_x, inset_y, inset_half):
    h, w, c = im.shape
    im_lo = cv2.resize(im, (w // blur, h // blur))
    im_lo_hi = cv2.resize(im_lo, (w, h))
    ymin = int(h * (inset_y - inset_half))
    ymax = int(h * (inset_y + inset_half))
    xmin = int(w * (inset_x - inset_half))
    xmax = int(w * (inset_x + inset_half))
    im_lo_hi[ymin:ymax, xmin:xmax] = im[ymin:ymax, xmin:xmax]
    return im_lo_hi

In [22]:
im = cv2.imread('data/im.jpg')  # (1080, 1920)
im_FF = blur_image(im, inset_x, inset_y, inset_half)
cv2.imwrite('data/im_FF.png', im_FF)

True

### Video - Fixed Foveation

In [ ]:
video_in = 'data/beatsaber.mp4'
video_out = 'beatsaber_FF.mp4'

cap = cv2.VideoCapture(video_in)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_out, fourcc, fps, (w, h))

while True:
    more, im = cap.read()
    if not more:
        break
    im_FF = blur_image(im, inset_x, inset_y, inset_half)
    out.write(im_FF)

### Video - Dynamic Foveation

In [32]:
inset_center_arr = []
inset_center = [inset_x, inset_y]

for i in range(100000):
    inset_velocity = [np.random.normal(scale=0.02), np.random.normal(scale=0.02)]
    inset_center[0] += inset_velocity[0]
    inset_center[1] += inset_velocity[1]
    inset_center[0] = max(min(inset_center[0], 1 - inset_half - 1/32), inset_half + 1/32)
    inset_center[1] = max(min(inset_center[1], 1 - inset_half - 1/32), inset_half + 1/32)
    inset_center_arr.append(inset_center.copy())

In [34]:
video_in = 'data/beatsaber.mp4'
video_out = 'beatsaber_DF.mp4'

cap = cv2.VideoCapture(video_in)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_out, fourcc, fps, (w, h))

for i in range(len(inset_center_arr)):
    more, im = cap.read()
    if not more:
        break
    inset_center = inset_center_arr[i]
    im_DF = blur_image(im, inset_center[0], inset_center[1], inset_half)
    im_DF[int(h * inset_center[1] - 2):int(h * inset_center[1] + 3), int(w * inset_center[0] - 2):int(w * inset_center[0] + 3), :] = 255
    out.write(im_DF)

cap.release()
out.release()
cv2.destroyAllWindows()

# Alpha Blending

In [5]:
im_FF = cv2.imread('data/im_FF.png')
im = cv2.imread('data/im.jpg')
h, w, c = im.shape
im_lo = cv2.resize(im, (w // blur, h // blur))
im_lo_hi = cv2.resize(im_lo, (w, h))

In [6]:
def smooth_alpha (im, im_lo_hi, inset_x, inset_y, inset_half, strip_half):
    im_out = im.copy()
    h, w, c = im.shape
    l, r = int(w * (inset_x - inset_half)), int(w * (inset_x + inset_half))
    u, d = int(h * (inset_y - inset_half)), int(h * (inset_y + inset_half))
    
    U = im[(u - strip_half):(u + strip_half + 1), l:r]
    D = im[(d - strip_half):(d + strip_half + 1), l:r]
    L = im[u:d, (l - strip_half):(l + strip_half + 1)]
    R = im[u:d, (r - strip_half):(r + strip_half + 1)]
    
    U_blur = im_lo_hi[(u - strip_half):(u + strip_half + 1), l:r]
    D_blur = im_lo_hi[(d - strip_half):(d + strip_half + 1), l:r]
    L_blur = im_lo_hi[u:d, (l - strip_half):(l + strip_half + 1)]
    R_blur = im_lo_hi[u:d, (r - strip_half):(r + strip_half + 1)]
    
    alpha_h = np.concatenate([np.linspace(0.0, 1.0, num=strip_half+1),
                              np.linspace(1.0, 0.0, num=strip_half+1)[1:]], axis=0).reshape(-1, 1, 1)
    alpha_v = alpha_h.reshape(1, -1, 1)
    im_out[(u - strip_half):(u + strip_half + 1), l:r] = alpha_h * U_blur + (1.0 - alpha_h) * U
    im_out[(d - strip_half):(d + strip_half + 1), l:r] = alpha_h * D_blur + (1.0 - alpha_h) * D
    im_out[u:d, (l - strip_half):(l + strip_half + 1)] = alpha_v * L_blur + (1.0 - alpha_v) * L
    im_out[u:d, (r - strip_half):(r + strip_half + 1)] = alpha_v * R_blur + (1.0 - alpha_v) * R
    
    return im_out

In [7]:
strip_half = 8
im_FF_alpha = smooth_alpha(im_FF, im_lo_hi, inset_x, inset_y, inset_half, strip_half)
cv2.imwrite('result/im_FF_alpha.png', im_FF_alpha)

True

# Gaussian

In [8]:
im_FF = cv2.imread('data/im_FF.png')

In [9]:
def smooth_gaussian2d (im, inset_x, inset_y, inset_half, sigma, kernel_half, strip_half, alpha=False):
    im_out = im.copy()
    h, w, c = im.shape
    l, r = int(w * (inset_x - inset_half)), int(w * (inset_x + inset_half))
    u, d = int(h * (inset_y - inset_half)), int(h * (inset_y + inset_half))
    
    window_size = kernel_half * 2 + 1
    gaussian_kernel_1d = signal.gaussian(window_size, std=sigma).reshape(window_size, 1)
    gaussian_kernel_1d /= np.sum(gaussian_kernel_1d)  # normalize
    gaussian_kernel_2d = np.outer(gaussian_kernel_1d, gaussian_kernel_1d)
    kernel = gaussian_kernel_2d
    
    U = im[(u - strip_half):(u + strip_half + 1), l:r]
    D = im[(d - strip_half):(d + strip_half + 1), l:r]
    L = im[u:d, (l - strip_half):(l + strip_half + 1)]
    R = im[u:d, (r - strip_half):(r + strip_half + 1)]
    
    U_blur = cv2.filter2D(U, -1, kernel)
    D_blur = cv2.filter2D(D, -1, kernel)
    L_blur = cv2.filter2D(L, -1, kernel)
    R_blur = cv2.filter2D(R, -1, kernel)
    
    if not alpha:
        im_out[(u - strip_half):(u + strip_half + 1), l:r] = U_blur
        im_out[(d - strip_half):(d + strip_half + 1), l:r] = D_blur
        im_out[u:d, (l - strip_half):(l + strip_half + 1)] = L_blur
        im_out[u:d, (r - strip_half):(r + strip_half + 1)] = R_blur
    else:
        alpha_h = np.concatenate([np.linspace(0.0, 1.0, num=strip_half+1),
                              np.linspace(1.0, 0.0, num=strip_half+1)[1:]], axis=0).reshape(-1, 1, 1)
        alpha_v = alpha_h.reshape(1, -1, 1)
        im_out[(u - strip_half):(u + strip_half + 1), l:r] = alpha_h * U_blur + (1.0 - alpha_h) * U
        im_out[(d - strip_half):(d + strip_half + 1), l:r] = alpha_h * D_blur + (1.0 - alpha_h) * D
        im_out[u:d, (l - strip_half):(l + strip_half + 1)] = alpha_v * L_blur + (1.0 - alpha_v) * L
        im_out[u:d, (r - strip_half):(r + strip_half + 1)] = alpha_v * R_blur + (1.0 - alpha_v) * R
    
    return im_out

In [10]:
def smooth_gaussian1d (im, inset_x, inset_y, inset_half, sigma, kernel_half, strip_half, alpha=False):
    im_out = im.copy()
    h, w, c = im.shape
    l, r = int(w * (inset_x - inset_half)), int(w * (inset_x + inset_half))
    u, d = int(h * (inset_y - inset_half)), int(h * (inset_y + inset_half))
    
    window_size = kernel_half * 2 + 1
    gaussian_kernel_1d = signal.gaussian(window_size, std=sigma).reshape(window_size, 1)
    gaussian_kernel_1d /= np.sum(gaussian_kernel_1d)
    kernel_h = gaussian_kernel_1d  # a vertical kernel for horizontal boundaries
    kernel_v = gaussian_kernel_1d.T  # a horizontal kernel for vertical boundaries
    
    U = im[(u - strip_half):(u + strip_half + 1), l:r]
    D = im[(d - strip_half):(d + strip_half + 1), l:r]
    L = im[u:d, (l - strip_half):(l + strip_half + 1)]
    R = im[u:d, (r - strip_half):(r + strip_half + 1)]
    
    U_blur = cv2.filter2D(U, -1, kernel_h)
    D_blur = cv2.filter2D(D, -1, kernel_h)
    L_blur = cv2.filter2D(L, -1, kernel_v)
    R_blur = cv2.filter2D(R, -1, kernel_v)
    
    if not alpha:
        im_out[(u - strip_half):(u + strip_half + 1), l:r] = U_blur
        im_out[(d - strip_half):(d + strip_half + 1), l:r] = D_blur
        im_out[u:d, (l - strip_half):(l + strip_half + 1)] = L_blur
        im_out[u:d, (r - strip_half):(r + strip_half + 1)] = R_blur
    else:
        alpha_h = np.concatenate([np.linspace(0.0, 1.0, num=strip_half+1),
                              np.linspace(1.0, 0.0, num=strip_half+1)[1:]], axis=0).reshape(-1, 1, 1)
        alpha_v = alpha_h.reshape(1, -1, 1)
        im_out[(u - strip_half):(u + strip_half + 1), l:r] = alpha_h * U_blur + (1.0 - alpha_h) * U
        im_out[(d - strip_half):(d + strip_half + 1), l:r] = alpha_h * D_blur + (1.0 - alpha_h) * D
        im_out[u:d, (l - strip_half):(l + strip_half + 1)] = alpha_v * L_blur + (1.0 - alpha_v) * L
        im_out[u:d, (r - strip_half):(r + strip_half + 1)] = alpha_v * R_blur + (1.0 - alpha_v) * R
    
    return im_out

In [ ]:
sigma = 1.0
kernel_half = 4
strip_half = 8

In [12]:
im_FF_gaussian2d = smooth_gaussian2d(im_FF, inset_x, inset_y, inset_half, sigma, kernel_half, strip_half)
cv2.imwrite('result/im_FF_gaussian2d.png', im_FF_gaussian2d)
im_FF_gaussian1d = smooth_gaussian1d(im_FF, inset_x, inset_y, inset_half, sigma, kernel_half, strip_half)
cv2.imwrite('result/im_FF_gaussian1d.png', im_FF_gaussian1d)

True

# Gaussian + Alpha Blending

In [13]:
im_FF_gaussian2d_alpha = smooth_gaussian2d(im_FF, inset_x, inset_y, inset_half, sigma, kernel_half, strip_half, alpha=True)
cv2.imwrite('result/im_FF_gaussian2d_alpha.png', im_FF_gaussian2d_alpha)
im_FF_gaussian1d_alpha = smooth_gaussian1d(im_FF, inset_x, inset_y, inset_half, sigma, kernel_half, strip_half, alpha=True)
cv2.imwrite('result/im_FF_gaussian1d_alpha.png', im_FF_gaussian1d_alpha)

True

# Laplacian Pyramid Blending

# Video

In [35]:
video_in = 'data/beatsaber_FF.mp4'
video_out = 'beatsaber_FF_gaussian1d_alpha.mp4'

cap = cv2.VideoCapture(video_in)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_out, fourcc, fps, (w, h))

while True:
    more, im = cap.read()
    if not more:
        break
    im_smooth = smooth_gaussian1d(im, inset_center[0], inset_center[1], inset_half, sigma, kernel_half, strip_half, alpha=True)
    out.write(im_smooth)

cap.release()
out.release()
cv2.destroyAllWindows()